# Sample Queries on the 1000 Genomes, gnomAD and ClinVar data lake

In this notebook, we will demonstrate some sample genomics queries that are typically made by clinical geneticists and researchers on genomics variant data. We will use the parquet/ORC transformed variant data from the 3502 DRAGEN-reanalyzed 1000 Genomes dataset available in 3 different schemas at s3://1000genomes-DRAGEN-data-lake-ready/:

1. var_partby_samples - Dataset partitioned by sample ID
2. var_partby_chrom - Dataset partitioned by chromosome and bucketed by samples
3. var_nested - Nested schema consisting of variant sites with sample IDs and genotypes that contain the variant

We use the annotations from ClinVar that are available at https://registry.opendata.aws/clinvar/ to demonstrate how to make queries that use the raw variant data with annotations.

## Import Dependencies

In [1]:
import boto3, os

s3 = boto3.resource('s3')
glue = boto3.client('glue')
cfn = boto3.client('cloudformation')
import sys
!{sys.executable} -m pip install PyAthena
from pyathena import connect
import pandas as pd
from pyathena.pandas.util import as_pandas

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
session = boto3.session.Session()
region = session.region_name
print(region)

us-east-1


In [28]:
databasename="\"1kg_full\""
partitioned_chr = "\"var_sortby\""
partitioned_samples = "\"var_partby_samples\""
nested = "\"var_nested\""

### Connect to Athena

In [43]:
conn = connect(s3_staging_dir='s3://athena-query-results-ss',region_name=region) #replace with your own bucket name


#### Let us explore the schema of the tables under the 1000 genomes transformed dataset

Here is the flat table schema for the data sorted by sample ID

In [44]:
cursor.execute("SELECT * from " + databasename + "." + partitioned_samples + "limit 10")
df = as_pandas(cursor)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 41 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   variant_id                     10 non-null     object 
 1   chrom                          10 non-null     object 
 2   pos                            10 non-null     int64  
 3   alleles                        10 non-null     object 
 4   rsid                           0 non-null      object 
 5   qual                           10 non-null     float64
 6   filters                        10 non-null     object 
 7   info.ac                        10 non-null     object 
 8   info.af                        10 non-null     object 
 9   info.an                        10 non-null     int64  
 10  info.db                        10 non-null     bool   
 11  info.dp                        10 non-null     int64  
 12  info.end                       0 non-null      object

Here is the schema for the flat table partitioned by chromosome and bucketed by samples. As you can see, the schema is very similar to the one partitioned by samples, except that "chrom" is the field that the data is partitioned by

In [30]:
cursor.execute("SELECT * from " + databasename + "." + partitioned_chr + "limit 10")
df = as_pandas(cursor)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   variant_id                     10 non-null     object 
 1   pos                            10 non-null     int64  
 2   ref                            10 non-null     object 
 3   alt                            10 non-null     object 
 4   sample_id                      10 non-null     object 
 5   alleles                        10 non-null     object 
 6   rsid                           0 non-null      object 
 7   qual                           10 non-null     float64
 8   filters                        10 non-null     object 
 9   info.ac                        0 non-null      object 
 10  info.af                        0 non-null      object 
 11  info.an                        0 non-null      object 
 12  info.db                        10 non-null     bool  

Here is the nested schema. In this schema, most of the FORMAT and INFO fields are not retained. Each row is a variant site with an array consisting of sample IDs and genotypes

In [31]:
cursor.execute("SELECT * from " + databasename +"." + nested + " limit 10")
df = as_pandas(cursor)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   variant_id  10 non-null     object
 1   pos         10 non-null     int64 
 2   ref         10 non-null     object
 3   alt         10 non-null     object
 4   samples     10 non-null     object
 5   chrom       10 non-null     object
dtypes: int64(1), object(5)
memory usage: 608.0+ bytes


,variant_id,pos,ref,alt,samples,chrom
0,chrY:G:C:2949388,2949388,G,C,"[{id=NA18504, gts=[1]}, {id=HG03812, gts=[1]},...",chrY
1,chrY:A:T:4634780,4634780,A,T,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
2,chrY:C:T:4758611,4758611,C,T,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
3,chrY:C:G:4806400,4806400,C,G,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
4,chrY:C:CGTATATATATGTGTATATATATACGTGTATATATACAT...,4896733,C,CGTATATATATGTGTATATATATACGTGTATATATACAT,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
5,chrY:C:CTA:5667108,5667108,C,CTA,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
6,chrY:T:G:6792036,6792036,T,G,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
7,chrY:T:TTG:7524147,7524147,T,TTG,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
8,chrY:T:TG:7664244,7664244,T,TG,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY
9,chrY:T:TA:8067814,8067814,T,TA,"[{id=NA18504, gts=[1]}, {id=HG01986, gts=[1]},...",chrY


### Queries on variant data alone
Here are some examples of queries on just the variant data with no annotations

**Example 1:** Query for variants that are on a specific gene BRCA1 (chr17:43044295-43125364) in a specific sample HG02625.
This query will run faster and will need to scan only data within the specific sample partition, so we will use the dataset that is partitioned by sample

In [35]:
cursor.execute("SELECT * from" + databasename + "." + partitioned_samples +
  "WHERE chrom='chr17'" 
  "AND pos BETWEEN 43044295 AND 43125364 "
  "AND sample_id = 'HG02625' ")

df=as_pandas(cursor)
df

,variant_id,chrom,pos,alleles,rsid,qual,filters,info.ac,info.af,info.an,...,mb,pl,pri,ps,sb,sq,sample_id,ref,alt,partition_0
0,chr17:G:A:43044391,chr17,43044391,"[G, A]",None,45.73,[],[1],[0.5],2,...,"[12, 17, 9, 10]","[81, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[13, 16, 13, 6]",None,HG02625,G,A,HG02625
1,chr17:CTT:C:43044804,chr17,43044804,"[CTT, C, CTTTT]",None,308.73,[],"[1, 1]","[0.5, 0.5]",2,...,"[0, 0, 17, 10]","[317, 263, 52, 391, 0, 54]","[0.0, 4.0, 7.0, 4.0, 8.0, 7.0]",NaN,"[0, 0, 17, 10]",None,HG02625,CTT,C,HG02625
2,chr17:C:A:43045257,chr17,43045257,"[C, A]",None,49.94,[],[1],[0.5],2,...,"[9, 10, 11, 7]","[85, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[12, 7, 8, 10]",None,HG02625,C,A,HG02625
3,chr17:A:G:43046604,chr17,43046604,"[A, G]",None,48.94,[],[1],[0.5],2,...,"[9, 8, 5, 8]","[84, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[11, 6, 9, 4]",None,HG02625,A,G,HG02625
4,chr17:C:CA:43046757,chr17,43046757,"[C, CA]",None,50.00,[],[1],[0.5],2,...,"[8, 5, 9, 6]","[56, 0, 50]","[0.0, 6.0, 9.0]",NaN,"[7, 6, 10, 5]",None,HG02625,C,CA,HG02625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,chr17:GTT:G:43123349,chr17,43123349,"[GTT, G]",None,50.00,[],[1],[0.5],2,...,"[3, 5, 7, 3]","[52, 0, 50]","[0.0, 2.0, 5.0]",NaN,"[5, 3, 4, 6]",None,HG02625,GTT,G,HG02625
207,chr17:A:G:43123628,chr17,43123628,"[A, G]",None,47.93,[],[1],[0.5],2,...,"[10, 12, 6, 10]","[83, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[12, 10, 8, 8]",None,HG02625,A,G,HG02625
208,chr17:A:G:43124230,chr17,43124230,"[A, G]",None,50.00,[],[1],[0.5],2,...,"[9, 7, 6, 12]","[85, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[8, 8, 13, 5]",None,HG02625,A,G,HG02625
209,chr17:T:C:43124331,chr17,43124331,"[T, C]",None,50.00,[],[1],[0.5],2,...,"[6, 5, 6, 6]","[85, 0, 50]","[0.0, 34.77, 37.77]",NaN,"[10, 1, 8, 4]",None,HG02625,T,C,HG02625


**Example 2:** Query to find samples that have variants in a specific region, in this case, the BRCA1 gene (chr17:43044295-43125364). 
This query can be run on the data that is partitioned by chromosome 

In [6]:
cursor.execute("SELECT DISTINCT(sample_id) from" + databasename + "." + var_sortby\" "
  "where chrom='chr17'" 
  "and pos between 43044295 and 43125364 ")

df=as_pandas(cursor)
df

,sample_id
0,HG03376
1,HG02635
2,NA18995
3,HG02293
4,HG01260
...,...
3197,NA19675
3198,HG02681
3199,HG03054
3200,HG00308


**Example 2a:** Query 2 on the nested data

In [36]:
cursor.execute("SELECT DISTINCT(sample.id) from ("
  "SELECT samples from" + databasename +  "." + nested + 
  "where chrom='chr1'" 
  "and pos between 9033567 and 9142000"
  ") as f," 
  "unnest(f.samples) as s(sample)"
  "order by 1")
df=as_pandas(cursor)
df

,id
0,HG00096
1,HG00097
2,HG00099
3,HG00100
4,HG00101
...,...
3197,NA21137
3198,NA21141
3199,NA21142
3200,NA21143


**Example 3:** Query to find only samples that have homozygous variants in a specific region, in this case, the BRCA1 gene (chr17:43044295-43125364). 

In [37]:
cursor.execute("SELECT DISTINCT(sample_id) from" + databasename + "." + partitioned_chr +
  "where chrom='chr17'" 
  "and pos between 43044295 and 43125364 "
  "and array_join(gt.alleles, '|') in ('0|1')")            )

df=as_pandas(cursor)
df

SyntaxError: invalid syntax (<ipython-input-37-66281c0b3a8e>, line 4)

### Queries joining variant data with ClinVar annotation data

**ClinVar has a number of tables, but the one that we will use here is the summary_variants table. This table has many fields, but we will only be using a subset of them. We also need to create a Variant ID that can be used to join with the raw variant tables, so to make this easier, we will create a "view" of the summary_variants table that we can use with subsequent queries**

In [38]:
cursor.execute("create or replace view clinvar as "
       "select concat('chr', chromosome, ':', referenceallelevcf, ':', alternateallelevcf, ':', cast(positionvcf as varchar)) as variant_id, "
       "chromosome as chrom, "
       "positionvcf as pos, "
       "referenceallelevcf as ref, "
       "alternateallelevcf as alt, "
       "genesymbol as genename, " 
       "clinicalsignificance as clinicalsignificance, "
       "numbersubmitters as num_submitters, "
       "reviewstatus as reviewstatus, "
       "split(phenotypeids, ',') as phenotypeids, "
       "split(phenotypelist, ',') as phenotypelist "
       "from \"clinvar_summary_variants_dl-awsroda\".\"variant_summary\" where assembly = 'GRCh38'"
        "and referenceallelevcf <> 'na' and alternateallelevcf <> 'na'")


**Example 4:** Modify query 3 to find the number of samples that have "variants of Unknown Significance" in the BRCA1 gene

In [39]:
cursor.execute("SELECT count(DISTINCT(sample.id)) FROM ("
       "SELECT samples FROM " + databasename + "." + nested + " AS v " 
       "JOIN clinvar a "
       "ON v.variant_id = a.variant_id "
       "WHERE a.clinicalsignificance = 'Uncertain significance' "
       "AND v.chrom='chr17' "
       "AND v.pos BETWEEN 43044295 AND 43125364 "
       ") AS f, "
       "UNNEST(f.samples) AS s(sample) ")
df=as_pandas(cursor)
df

,_col0
0,1550


**Example 4a:** To have a greater degree of confidence in the results, we want to only consider those entries in ClinVar that have more than 1 submission. This query will filter the results from Example 4 to only consider those that have > 1 submitter.

In [40]:
cursor.execute("SELECT count(DISTINCT(sample.id)) from ("
       "SELECT samples FROM" + databasename + "." + nested + " as v " 
       "JOIN clinvar a "
       "ON v.variant_id = a.variant_id "
       "WHERE a.clinicalsignificance = 'Uncertain significance' "
       "AND v.chrom='chr17' "
       "AND v.pos BETWEEN 43044295 AND 43125364 "
       "AND a.num_submitters > 1 "
       ") AS f, "
       "UNNEST(f.samples) AS s(sample) ")
df=as_pandas(cursor)
df

,_col0
0,41


**Example 5a:**  Find all variants in this dataset that have a pathogenic variant in the BRCA1 gene (using the gene annotation from ClinVar). Running the same query using the chromosome and position range for the BRCA1 gene is much faster due to the partitioning of the data.

In [41]:
cursor.execute("SELECT COUNT(DISTINCT sample_id) FROM " + databasename + "." + partitioned_chr + " as v " 
               "JOIN clinvar AS a " 
               "ON v.variant_id = a.variant_id " 
               "WHERE a.genename='BRCA1' AND clinicalsignificance='Pathogenic' ")

df=as_pandas(cursor)
df

,_col0
0,4


**Example 5b:** Run Query 5a using the nested dataset

In [42]:
cursor.execute("SELECT COUNT(DISTINCT sample.id) FROM " 
               "(SELECT samples from " + databasename + "." + nested + " AS v " 
               "JOIN clinvar AS a " 
               "ON v.variant_id = a.variant_id " 
               "WHERE a.genename='BRCA1' AND clinicalsignificance='Pathogenic') AS f, "
               "UNNEST(f.samples) AS s(sample)")

df=as_pandas(cursor)
df

,_col0
0,4
